# The Math of Backpropagation

*Following the gradient home.*

---

We're going to trace backpropagation through our bag-of-words model from notebook 01. The model is simple enough that we can compute every gradient by hand, then verify with PyTorch.

**The model:**
```
tokens → Embedding → Average → Linear → Softmax → Loss
```

**Our goal:** Compute ∂loss/∂everything. By the end, you'll understand:
1. What the chain rule actually does
2. How gradients flow backward through each operation
3. Why this is called "backpropagation"
4. What `.backward()` is actually computing

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Reproducibility
torch.manual_seed(42)

# We'll work with tiny dimensions so we can see everything
print("Ready to learn.")

Ready to learn.


---

## 0. The Big Picture: What Is Backpropagation?

Training a neural network means adjusting weights to minimize loss. To know *how* to adjust them, we need gradients—the derivative of the loss with respect to each weight.

**The problem:** Our network is a composition of functions:
$$\text{loss} = f_5(f_4(f_3(f_2(f_1(\text{input})))))$$

How do we compute $\frac{\partial \text{loss}}{\partial W}$ for some weight $W$ buried deep in $f_2$?

**The answer:** The chain rule.

### The Chain Rule (Single Variable)

If $y = f(g(x))$, then:
$$\frac{dy}{dx} = \frac{dy}{dg} \cdot \frac{dg}{dx}$$

You multiply the derivatives along the path.

### The Chain Rule (Many Variables)

If $L$ depends on $y$, and $y$ depends on $x$, then:
$$\frac{\partial L}{\partial x} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial x}$$

With vectors/matrices, these become Jacobians, but the principle is the same: **gradients flow backward through the computation graph, multiplying at each step.**

### Why "Back" Propagation?

Because we start at the end (the loss) and work backward. At each step, we ask: "Given how the loss changes with respect to this layer's *output*, how does it change with respect to this layer's *input* and *parameters*?"

That's the whole algorithm. Let's see it in action.

---

## 1. Our Model: A Tiny Bag of Words

Let's build the smallest possible version of our amoeba:
- Vocabulary: 4 tokens
- Embedding dimension: 3
- Input sequence: 2 tokens

Small enough to trace every number by hand.

In [2]:
# === Tiny dimensions ===
vocab_size = 4
d_model = 3
seq_len = 2

# === Initialize weights manually ===
# (So we know exactly what they are)

# Embedding matrix: vocab_size × d_model
# Each row is one token's embedding
W_embed = torch.tensor([
    [1.0, 0.0, 0.0],   # Token 0
    [0.0, 1.0, 0.0],   # Token 1  
    [0.0, 0.0, 1.0],   # Token 2
    [1.0, 1.0, 1.0],   # Token 3
], requires_grad=True)

# Unembed matrix: d_model × vocab_size (we'll use W @ x, so it's transposed from nn.Linear)
# For simplicity, let's make it the transpose of embedding (no bias)
W_unembed = torch.tensor([
    [1.0, 0.0, 0.0, 0.5],
    [0.0, 1.0, 0.0, 0.5],
    [0.0, 0.0, 1.0, 0.5],
], requires_grad=True)

print(f"W_embed shape: {W_embed.shape}")
print(f"W_unembed shape: {W_unembed.shape}")

W_embed shape: torch.Size([4, 3])
W_unembed shape: torch.Size([3, 4])


In [3]:
# === Input ===
# A sequence of 2 tokens: [token 1, token 2]
# Target: token 3

input_tokens = torch.tensor([1, 2])  # Token IDs
target = torch.tensor(3)             # What we want to predict

print(f"Input tokens: {input_tokens.tolist()}")
print(f"Target: {target.item()}")

Input tokens: [1, 2]
Target: 3


---

## 2. Forward Pass: Computing the Loss

Let's trace the forward pass step by step, naming every intermediate value.

### Step 1: Embedding Lookup
$$e_i = W_{\text{embed}}[\text{token}_i]$$

This is just indexing—grab the rows corresponding to our input tokens.

In [4]:
# Step 1: Embedding lookup
# For each token ID, grab that row from the embedding matrix

e = W_embed[input_tokens]  # Shape: [seq_len, d_model] = [2, 3]

print("Step 1: Embedding lookup")
print(f"  Token 1 → embedding: {e[0].tolist()}")
print(f"  Token 2 → embedding: {e[1].tolist()}")
print(f"  e shape: {e.shape}")
print(f"  e = ")
print(e.detach().numpy())

Step 1: Embedding lookup
  Token 1 → embedding: [0.0, 1.0, 0.0]
  Token 2 → embedding: [0.0, 0.0, 1.0]
  e shape: torch.Size([2, 3])
  e = 
[[0. 1. 0.]
 [0. 0. 1.]]


### Step 2: Average Pooling
$$h = \frac{1}{n} \sum_{i=1}^{n} e_i$$

We collapse the sequence into a single vector by averaging.

In [5]:
# Step 2: Average the embeddings
h = e.mean(dim=0)  # Shape: [d_model] = [3]

print("Step 2: Average pooling")
print(f"  h = mean([{e[0].tolist()}, {e[1].tolist()}])")
print(f"  h = {h.tolist()}")

Step 2: Average pooling
  h = mean([[0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
  h = [0.0, 0.5, 0.5]


### Step 3: Linear Projection (Unembed)
$$z = W_{\text{unembed}}^T h$$

Project the hidden state to vocabulary-sized logits.

In [6]:
# Step 3: Linear projection to logits
# z = W_unembed.T @ h (or equivalently, h @ W_unembed)

z = h @ W_unembed  # Shape: [vocab_size] = [4]

print("Step 3: Linear projection")
print(f"  z = h @ W_unembed")
print(f"  z = {z.tolist()}")
print(f"  (These are 'logits' — unnormalized scores for each token)")

Step 3: Linear projection
  z = h @ W_unembed
  z = [0.0, 0.5, 0.5, 0.5]
  (These are 'logits' — unnormalized scores for each token)


### Step 4: Softmax
$$p_i = \frac{e^{z_i}}{\sum_j e^{z_j}}$$

Turn logits into a probability distribution.

In [7]:
# Step 4: Softmax
p = F.softmax(z, dim=0)  # Shape: [vocab_size] = [4]

print("Step 4: Softmax")
print(f"  p = softmax(z)")
print(f"  p = {[f'{x:.4f}' for x in p.tolist()]}")
print(f"  Sum: {p.sum().item():.6f}")

Step 4: Softmax
  p = softmax(z)
  p = ['0.1682', '0.2773', '0.2773', '0.2773']
  Sum: 1.000000


### Step 5: Cross-Entropy Loss
$$L = -\log(p_{\text{target}})$$

How surprised are we by the correct answer? Lower probability → higher loss.

In [8]:
# Step 5: Cross-entropy loss
# L = -log(p[target])

L = -torch.log(p[target])

print("Step 5: Cross-entropy loss")
print(f"  Target: token {target.item()}")
print(f"  p[target] = p[{target.item()}] = {p[target].item():.4f}")
print(f"  L = -log({p[target].item():.4f}) = {L.item():.4f}")

Step 5: Cross-entropy loss
  Target: token 3
  p[target] = p[3] = 0.2773
  L = -log(0.2773) = 1.2827


In [9]:
# Summary of forward pass
print("=" * 50)
print("FORWARD PASS COMPLETE")
print("=" * 50)
print(f"\nInput tokens: {input_tokens.tolist()}")
print(f"Target: {target.item()}")
print(f"")
print(f"e (embeddings):     shape {e.shape}")
print(f"h (averaged):       {h.tolist()}")
print(f"z (logits):         {[f'{x:.3f}' for x in z.tolist()]}")
print(f"p (probabilities):  {[f'{x:.3f}' for x in p.tolist()]}")
print(f"L (loss):           {L.item():.4f}")

FORWARD PASS COMPLETE

Input tokens: [1, 2]
Target: 3

e (embeddings):     shape torch.Size([2, 3])
h (averaged):       [0.0, 0.5, 0.5]
z (logits):         ['0.000', '0.500', '0.500', '0.500']
p (probabilities):  ['0.168', '0.277', '0.277', '0.277']
L (loss):           1.2827


---

## 3. Backward Pass: The Chain Rule in Action

Now we go backward. At each step, we'll:
1. State the relationship (what depends on what)
2. Derive the gradient formula
3. Compute it numerically
4. Verify with PyTorch

### The Key Insight

At each layer, we receive $\frac{\partial L}{\partial \text{output}}$ from the layer above, and we compute:
- $\frac{\partial L}{\partial \text{input}}$ — to pass to the layer below
- $\frac{\partial L}{\partial W}$ — to update this layer's weights

This is why it's called back*propagation*—the gradient propagates backward.

### Step 5 (backward): ∂L/∂p

We have $L = -\log(p_{\text{target}})$.

The derivative of $-\log(x)$ is $-1/x$, so:

$$\frac{\partial L}{\partial p_i} = \begin{cases} -1/p_i & \text{if } i = \text{target} \\ 0 & \text{otherwise} \end{cases}$$

In [10]:
# Step 5 backward: ∂L/∂p

dL_dp = torch.zeros(vocab_size)
dL_dp[target] = -1.0 / p[target].item()

print("Step 5 backward: ∂L/∂p")
print(f"  ∂L/∂p = {[f'{x:.4f}' for x in dL_dp.tolist()]}")
print(f"  (Only position {target.item()} is non-zero)")

Step 5 backward: ∂L/∂p
  ∂L/∂p = ['0.0000', '0.0000', '0.0000', '-3.6065']
  (Only position 3 is non-zero)


### Step 4 (backward): ∂L/∂z (through softmax)

This is the trickiest part. Softmax has a coupling: every $z_j$ affects every $p_i$.

The Jacobian of softmax is:
$$\frac{\partial p_i}{\partial z_j} = p_i(\delta_{ij} - p_j)$$

Where $\delta_{ij}$ is 1 if $i=j$, else 0.

Using the chain rule:
$$\frac{\partial L}{\partial z_j} = \sum_i \frac{\partial L}{\partial p_i} \frac{\partial p_i}{\partial z_j}$$

**But here's a beautiful simplification.** For cross-entropy loss with softmax:
$$\frac{\partial L}{\partial z_i} = p_i - y_i$$

Where $y$ is the one-hot target vector. The prediction minus the truth. Elegant.

In [11]:
# Step 4 backward: ∂L/∂z
# For cross-entropy + softmax: ∂L/∂z = p - y (one-hot)

# Create one-hot target
y_onehot = torch.zeros(vocab_size)
y_onehot[target] = 1.0

# The gradient is just p - y
dL_dz = p.detach() - y_onehot

print("Step 4 backward: ∂L/∂z")
print(f"  p =     {[f'{x:.4f}' for x in p.detach().tolist()]}")
print(f"  y =     {[f'{x:.4f}' for x in y_onehot.tolist()]}")
print(f"  ∂L/∂z = {[f'{x:.4f}' for x in dL_dz.tolist()]}")
print(f"")
print(f"  Notice: for the target token ({target.item()}), gradient is p-1 = {p[target].item():.4f} - 1 = {dL_dz[target].item():.4f}")
print(f"  For other tokens, gradient equals their probability (pushing them down)")

Step 4 backward: ∂L/∂z
  p =     ['0.1682', '0.2773', '0.2773', '0.2773']
  y =     ['0.0000', '0.0000', '0.0000', '1.0000']
  ∂L/∂z = ['0.1682', '0.2773', '0.2773', '-0.7227']

  Notice: for the target token (3), gradient is p-1 = 0.2773 - 1 = -0.7227
  For other tokens, gradient equals their probability (pushing them down)


### Step 3 (backward): ∂L/∂h and ∂L/∂W_unembed

We had $z = h \cdot W_{\text{unembed}}$ (or $z_j = \sum_k h_k W_{kj}$).

**Gradient w.r.t. h:**
$$\frac{\partial L}{\partial h_k} = \sum_j \frac{\partial L}{\partial z_j} \frac{\partial z_j}{\partial h_k} = \sum_j \frac{\partial L}{\partial z_j} W_{kj}$$

In matrix form: $\frac{\partial L}{\partial h} = W_{\text{unembed}} \cdot \frac{\partial L}{\partial z}$

**Gradient w.r.t. W_unembed:**
$$\frac{\partial L}{\partial W_{kj}} = \frac{\partial L}{\partial z_j} \cdot h_k$$

In matrix form: $\frac{\partial L}{\partial W_{\text{unembed}}} = h^T \cdot \frac{\partial L}{\partial z}$ (outer product)

In [12]:
# Step 3 backward: ∂L/∂h and ∂L/∂W_unembed

# Gradient w.r.t. h
dL_dh = W_unembed.detach() @ dL_dz  # [d_model, vocab] @ [vocab] = [d_model]

# Gradient w.r.t. W_unembed (outer product)
dL_dW_unembed = torch.outer(h.detach(), dL_dz)  # [d_model] outer [vocab] = [d_model, vocab]

print("Step 3 backward: ∂L/∂h and ∂L/∂W_unembed")
print(f"\n  ∂L/∂h = W_unembed @ ∂L/∂z")
print(f"  ∂L/∂h = {[f'{x:.4f}' for x in dL_dh.tolist()]}")
print(f"\n  ∂L/∂W_unembed = outer(h, ∂L/∂z)")
print(f"  Shape: {dL_dW_unembed.shape}")
print(f"  ∂L/∂W_unembed = ")
print(dL_dW_unembed.numpy().round(4))

Step 3 backward: ∂L/∂h and ∂L/∂W_unembed

  ∂L/∂h = W_unembed @ ∂L/∂z
  ∂L/∂h = ['-0.1932', '-0.0841', '-0.0841']

  ∂L/∂W_unembed = outer(h, ∂L/∂z)
  Shape: torch.Size([3, 4])
  ∂L/∂W_unembed = 
[[ 0.      0.      0.     -0.    ]
 [ 0.0841  0.1386  0.1386 -0.3614]
 [ 0.0841  0.1386  0.1386 -0.3614]]


### Step 2 (backward): ∂L/∂e (through averaging)

We had $h = \frac{1}{n} \sum_i e_i$.

Each embedding contributes equally, so:
$$\frac{\partial L}{\partial e_i} = \frac{1}{n} \frac{\partial L}{\partial h}$$

The gradient is just copied to each position, scaled by 1/n.

In [13]:
# Step 2 backward: ∂L/∂e

n = seq_len
dL_de = dL_dh.unsqueeze(0).expand(n, -1) / n  # Copy to each position, scale by 1/n

print("Step 2 backward: ∂L/∂e")
print(f"  ∂L/∂e = (1/{n}) × ∂L/∂h broadcast to each position")
print(f"  Shape: {dL_de.shape}")
print(f"  ∂L/∂e[0] = {[f'{x:.4f}' for x in dL_de[0].tolist()]}")
print(f"  ∂L/∂e[1] = {[f'{x:.4f}' for x in dL_de[1].tolist()]}")
print(f"  (Both rows are the same — the gradient is distributed equally)")

Step 2 backward: ∂L/∂e
  ∂L/∂e = (1/2) × ∂L/∂h broadcast to each position
  Shape: torch.Size([2, 3])
  ∂L/∂e[0] = ['-0.0966', '-0.0420', '-0.0420']
  ∂L/∂e[1] = ['-0.0966', '-0.0420', '-0.0420']
  (Both rows are the same — the gradient is distributed equally)


### Step 1 (backward): ∂L/∂W_embed (through embedding lookup)

The embedding lookup selected rows from W_embed. The gradient only affects those rows:

$$\frac{\partial L}{\partial W_{\text{embed}}[i]} = \begin{cases} \frac{\partial L}{\partial e_j} & \text{if token } j \text{ = } i \\ 0 & \text{otherwise} \end{cases}$$

Only the rows we actually used get gradients.

In [14]:
# Step 1 backward: ∂L/∂W_embed

dL_dW_embed = torch.zeros_like(W_embed)

# Scatter the gradients to the rows that were used
for i, token_id in enumerate(input_tokens):
    dL_dW_embed[token_id] += dL_de[i]

print("Step 1 backward: ∂L/∂W_embed")
print(f"  Input tokens: {input_tokens.tolist()}")
print(f"  Only rows {input_tokens.tolist()} get gradients")
print(f"")
print(f"  ∂L/∂W_embed = ")
print(dL_dW_embed.numpy().round(4))
print(f"")
print(f"  Row 0 (token 0): {dL_dW_embed[0].tolist()} (not used)")
print(f"  Row 1 (token 1): {[f'{x:.4f}' for x in dL_dW_embed[1].tolist()]} (used once)")
print(f"  Row 2 (token 2): {[f'{x:.4f}' for x in dL_dW_embed[2].tolist()]} (used once)")
print(f"  Row 3 (token 3): {dL_dW_embed[3].tolist()} (not used)")

Step 1 backward: ∂L/∂W_embed
  Input tokens: [1, 2]
  Only rows [1, 2] get gradients

  ∂L/∂W_embed = 
[[ 0.      0.      0.    ]
 [-0.0966 -0.042  -0.042 ]
 [-0.0966 -0.042  -0.042 ]
 [ 0.      0.      0.    ]]

  Row 0 (token 0): [0.0, 0.0, 0.0] (not used)
  Row 1 (token 1): ['-0.0966', '-0.0420', '-0.0420'] (used once)
  Row 2 (token 2): ['-0.0966', '-0.0420', '-0.0420'] (used once)
  Row 3 (token 3): [0.0, 0.0, 0.0] (not used)


---

## 4. Verification: Compare with PyTorch Autograd

Now let's verify our manual gradients match what PyTorch computes with `.backward()`.

In [15]:
# Re-run forward pass with fresh tensors (gradients attached)
W_embed_v = torch.tensor([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [1.0, 1.0, 1.0],
], requires_grad=True)

W_unembed_v = torch.tensor([
    [1.0, 0.0, 0.0, 0.5],
    [0.0, 1.0, 0.0, 0.5],
    [0.0, 0.0, 1.0, 0.5],
], requires_grad=True)

# Forward pass
e_v = W_embed_v[input_tokens]
h_v = e_v.mean(dim=0)
z_v = h_v @ W_unembed_v
p_v = F.softmax(z_v, dim=0)
L_v = -torch.log(p_v[target])

print(f"Loss (verification): {L_v.item():.4f}")
print(f"Loss (original):     {L.item():.4f}")

Loss (verification): 1.2827
Loss (original):     1.2827


In [16]:
# Run backward pass
L_v.backward()

print("PyTorch autograd gradients:")
print(f"\n∂L/∂W_embed (PyTorch):")
print(W_embed_v.grad.numpy().round(4))
print(f"\n∂L/∂W_embed (manual):")
print(dL_dW_embed.numpy().round(4))

print(f"\n∂L/∂W_unembed (PyTorch):")
print(W_unembed_v.grad.numpy().round(4))
print(f"\n∂L/∂W_unembed (manual):")
print(dL_dW_unembed.numpy().round(4))

PyTorch autograd gradients:

∂L/∂W_embed (PyTorch):
[[ 0.      0.      0.    ]
 [-0.0966 -0.042  -0.042 ]
 [-0.0966 -0.042  -0.042 ]
 [ 0.      0.      0.    ]]

∂L/∂W_embed (manual):
[[ 0.      0.      0.    ]
 [-0.0966 -0.042  -0.042 ]
 [-0.0966 -0.042  -0.042 ]
 [ 0.      0.      0.    ]]

∂L/∂W_unembed (PyTorch):
[[ 0.      0.      0.      0.    ]
 [ 0.0841  0.1386  0.1386 -0.3614]
 [ 0.0841  0.1386  0.1386 -0.3614]]

∂L/∂W_unembed (manual):
[[ 0.      0.      0.     -0.    ]
 [ 0.0841  0.1386  0.1386 -0.3614]
 [ 0.0841  0.1386  0.1386 -0.3614]]


In [17]:
# Check they match
embed_match = torch.allclose(W_embed_v.grad, dL_dW_embed, atol=1e-5)
unembed_match = torch.allclose(W_unembed_v.grad, dL_dW_unembed, atol=1e-5)

print("=" * 50)
print("VERIFICATION")
print("=" * 50)
print(f"W_embed gradients match:   {embed_match} ✓" if embed_match else f"W_embed gradients match:   {embed_match} ✗")
print(f"W_unembed gradients match: {unembed_match} ✓" if unembed_match else f"W_unembed gradients match: {unembed_match} ✗")

VERIFICATION
W_embed gradients match:   True ✓
W_unembed gradients match: True ✓


---

## 5. Visualizing the Flow

Let's see the complete picture: forward values and backward gradients side by side.

In [18]:
print("=" * 60)
print("THE COMPLETE PICTURE")
print("=" * 60)
print()
print("FORWARD PASS (left to right)")
print("-" * 60)
print(f"tokens: {input_tokens.tolist()}")
print(f"   ↓ embedding lookup")
print(f"e: [{e[0].detach().tolist()}, {e[1].detach().tolist()}]")
print(f"   ↓ average")
print(f"h: {h.detach().tolist()}")
print(f"   ↓ linear (× W_unembed)")
print(f"z: {[f'{x:.3f}' for x in z.detach().tolist()]}")
print(f"   ↓ softmax")
print(f"p: {[f'{x:.3f}' for x in p.detach().tolist()]}")
print(f"   ↓ -log(p[target={target.item()}])")
print(f"L: {L.item():.4f}")
print()
print("BACKWARD PASS (right to left)")
print("-" * 60)
print(f"∂L/∂L: 1 (by definition)")
print(f"   ↓")
print(f"∂L/∂p: {[f'{x:.3f}' for x in dL_dp.tolist()]}")
print(f"   ↓ softmax + cross-entropy (combined)")
print(f"∂L/∂z: {[f'{x:.3f}' for x in dL_dz.tolist()]}  (= p - y_onehot)")
print(f"   ↓ linear")
print(f"∂L/∂h: {[f'{x:.3f}' for x in dL_dh.tolist()]}")
print(f"   ↓ average (divide by n={seq_len})")
print(f"∂L/∂e: [{[f'{x:.3f}' for x in dL_de[0].tolist()]}, {[f'{x:.3f}' for x in dL_de[1].tolist()]}]")
print(f"   ↓ scatter to used rows")
print(f"∂L/∂W_embed: (see matrix above)")
print()
print("WEIGHT GRADIENTS (what optimizer.step() uses)")
print("-" * 60)
print(f"∂L/∂W_embed shape:   {dL_dW_embed.shape}")
print(f"∂L/∂W_unembed shape: {dL_dW_unembed.shape}")

THE COMPLETE PICTURE

FORWARD PASS (left to right)
------------------------------------------------------------
tokens: [1, 2]
   ↓ embedding lookup
e: [[0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
   ↓ average
h: [0.0, 0.5, 0.5]
   ↓ linear (× W_unembed)
z: ['0.000', '0.500', '0.500', '0.500']
   ↓ softmax
p: ['0.168', '0.277', '0.277', '0.277']
   ↓ -log(p[target=3])
L: 1.2827

BACKWARD PASS (right to left)
------------------------------------------------------------
∂L/∂L: 1 (by definition)
   ↓
∂L/∂p: ['0.000', '0.000', '0.000', '-3.607']
   ↓ softmax + cross-entropy (combined)
∂L/∂z: ['0.168', '0.277', '0.277', '-0.723']  (= p - y_onehot)
   ↓ linear
∂L/∂h: ['-0.193', '-0.084', '-0.084']
   ↓ average (divide by n=2)
∂L/∂e: [['-0.097', '-0.042', '-0.042'], ['-0.097', '-0.042', '-0.042']]
   ↓ scatter to used rows
∂L/∂W_embed: (see matrix above)

WEIGHT GRADIENTS (what optimizer.step() uses)
------------------------------------------------------------
∂L/∂W_embed shape:   torch.Size([4, 3])
∂L

---

## Summary: What Backpropagation Actually Does

### The Algorithm

1. **Forward pass:** Compute all intermediate values, building a computation graph.

2. **Backward pass:** Starting from the loss, propagate gradients backward:
   - At each node, receive ∂L/∂output from above
   - Compute ∂L/∂input (to pass backward) and ∂L/∂weights (to update parameters)
   - The chain rule connects them: ∂L/∂input = ∂L/∂output × ∂output/∂input

3. **Update:** Use the weight gradients to adjust parameters toward lower loss.

### Key Insights

- **Gradients flow backward** through the same path values flowed forward
- **The chain rule** multiplies gradients at each step
- **Only used weights get gradients** (sparse updates for embedding lookups)
- **Cross-entropy + softmax simplifies beautifully:** ∂L/∂z = p - y

### What PyTorch Does Automatically

Every operation in PyTorch records:
- What inputs it received
- How to compute ∂output/∂input

When you call `.backward()`, it walks this graph in reverse, applying the chain rule at each step. That's it. That's the magic.

---

*Now when you see `loss.backward()`, you know what's happening inside.*